<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#최종평가-:-category-classification-evaluation" data-toc-modified-id="최종평가-:-category-classification-evaluation-1">최종평가 : category classification evaluation</a></span><ul class="toc-item"><li><span><a href="#과제-내용" data-toc-modified-id="과제-내용-1.1">과제 내용</a></span><ul class="toc-item"><li><span><a href="#[classification]-:-프로젝트3(kNN),-프로젝트4(Naive-Bayes)" data-toc-modified-id="[classification]-:-프로젝트3(kNN),-프로젝트4(Naive-Bayes)-1.1.1">[classification] : 프로젝트3(kNN), 프로젝트4(Naive Bayes)</a></span></li><li><span><a href="#[Evlauation]-:-F-Measure" data-toc-modified-id="[Evlauation]-:-F-Measure-1.1.2">[Evlauation] : F-Measure</a></span></li></ul></li><li><span><a href="#문제-풀이" data-toc-modified-id="문제-풀이-1.2">문제 풀이</a></span><ul class="toc-item"><li><span><a href="#package-import" data-toc-modified-id="package-import-1.2.1">package import</a></span></li><li><span><a href="#학습" data-toc-modified-id="학습-1.2.2">학습</a></span><ul class="toc-item"><li><span><a href="#수집한-문서목록-가져오기" data-toc-modified-id="수집한-문서목록-가져오기-1.2.2.1">수집한 문서목록 가져오기</a></span></li><li><span><a href="#각-문서에서-색인어-추출-전처리" data-toc-modified-id="각-문서에서-색인어-추출-전처리-1.2.2.2">각 문서에서 색인어 추출 전처리</a></span></li><li><span><a href="#train-set,-test-set-분리" data-toc-modified-id="train-set,-test-set-분리-1.2.2.3">train set, test set 분리</a></span></li><li><span><a href="#Inverted-Index-:-DTM" data-toc-modified-id="Inverted-Index-:-DTM-1.2.2.4">Inverted Index : DTM</a></span></li><li><span><a href="#DTM" data-toc-modified-id="DTM-1.2.2.5">DTM</a></span></li><li><span><a href="#TDM" data-toc-modified-id="TDM-1.2.2.6">TDM</a></span></li><li><span><a href="#TWM" data-toc-modified-id="TWM-1.2.2.7">TWM</a></span></li><li><span><a href="#Evaluate-IDF" data-toc-modified-id="Evaluate-IDF-1.2.2.8">Evaluate IDF</a></span></li></ul></li><li><span><a href="#Validation" data-toc-modified-id="Validation-1.2.3">Validation</a></span><ul class="toc-item"><li><span><a href="#Naive-Bayes-Validation" data-toc-modified-id="Naive-Bayes-Validation-1.2.3.1">Naive Bayes Validation</a></span></li></ul></li><li><span><a href="#test-data-성능평가" data-toc-modified-id="test-data-성능평가-1.2.4">test data 성능평가</a></span><ul class="toc-item"><li><span><a href="#test-data-읽어오기" data-toc-modified-id="test-data-읽어오기-1.2.4.1">test data 읽어오기</a></span></li><li><span><a href="#Test-Data-Prediction" data-toc-modified-id="Test-Data-Prediction-1.2.4.2">Test Data Prediction</a></span></li><li><span><a href="#성능-평가" data-toc-modified-id="성능-평가-1.2.4.3">성능 평가</a></span></li></ul></li></ul></li></ul></li></ul></div>

# 최종평가 : category classification evaluation

## 과제 내용

### [classification] : 프로젝트3(kNN), 프로젝트4(Naive Bayes)

1. 뉴스 기사 분류 (kNN)
2. 뉴스 기사 분류 (Naive Bayes)

### [Evlauation] : F-Measure

3. kNN : k값을 1, 3, 5, 7, 9로 변경하면서 Precision, Recall, F-Measure 평가
4. Naive Bayes : 뉴스 기사를 traing set과 validation set으로 분리하여 Precision, Recall, F-Measure 평가

## 문제 풀이

### package import

In [1]:
import os
from collections import defaultdict
from math import log
from math import exp

In [2]:
import numpy as np

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [3]:
from functions.naver import NewsScraping

from functions.info_retrieval import max_tf, raw_idf
from functions.info_retrieval import smoothig_idf
from functions.info_retrieval import clean_collection
from functions.info_retrieval import extend_lexicon
from functions.info_retrieval import inverted_index_with_tf
from functions.info_retrieval import get_tdm_from_dtm
from functions.info_retrieval import tdm2twm
from functions.info_retrieval import evaluate_idf

### 학습

In [4]:
category = {
    '정치': '00',
    '경제': '01',
    '사회': '02',
    '생활문화': '03',
    '세계': '04',
    'IT과학': '05'
}

In [5]:
inverse_category = {
    '00': '정치',
    '01': '경제',
    '02': '사회',
    '03': '생활문화',
    '04': '세계',
    '05': 'IT과학'
}

#### 수집한 문서목록 가져오기

In [6]:
news = NewsScraping()

In [7]:
filenames = news.get_filenames(all_folder=True)
len(filenames)

540

In [8]:
collection = []   # tuple(filename, content)들의 list

for filename in filenames:
    collection.append((filename.split("/")[-1], news.get_content(filename)))

In [9]:
len(collection)

540

In [10]:
#collection[:3]

#### 각 문서에서 색인어 추출 전처리

- 불용어 제거
- 어절 -> 형태소 + 명사 + 바이그램

In [11]:
cleaned_collection = clean_collection(collection)

In [12]:
#cleaned_collection[:3]

In [13]:
extended_collection = []
corpus_count = len(cleaned_collection)

for idx, (filename, corpus) in enumerate(cleaned_collection):
    extended_lexicon = extend_lexicon(corpus)
    extended_collection.append((filename, extended_lexicon))
    if (idx + 1) % 30 == 0:
        print(corpus_count, "개 documents 중", idx + 1, "개 documents의 lexicon 추출 완료")

print(idx + 1, "개 documents의 lexicon 추출 완료")

540 개 documents 중 30 개 documents의 lexicon 추출 완료
540 개 documents 중 60 개 documents의 lexicon 추출 완료
540 개 documents 중 90 개 documents의 lexicon 추출 완료
540 개 documents 중 120 개 documents의 lexicon 추출 완료
540 개 documents 중 150 개 documents의 lexicon 추출 완료
540 개 documents 중 180 개 documents의 lexicon 추출 완료
540 개 documents 중 210 개 documents의 lexicon 추출 완료
540 개 documents 중 240 개 documents의 lexicon 추출 완료
540 개 documents 중 270 개 documents의 lexicon 추출 완료
540 개 documents 중 300 개 documents의 lexicon 추출 완료
540 개 documents 중 330 개 documents의 lexicon 추출 완료
540 개 documents 중 360 개 documents의 lexicon 추출 완료
540 개 documents 중 390 개 documents의 lexicon 추출 완료
540 개 documents 중 420 개 documents의 lexicon 추출 완료
540 개 documents 중 450 개 documents의 lexicon 추출 완료
540 개 documents 중 480 개 documents의 lexicon 추출 완료
540 개 documents 중 510 개 documents의 lexicon 추출 완료
540 개 documents 중 540 개 documents의 lexicon 추출 완료
540 개 documents의 lexicon 추출 완료


In [14]:
#extended_collection[0:3]

#### train set, test set 분리

In [15]:
data_set = extended_collection
target = [filename.split("-")[0] for filename, content in extended_collection]

In [16]:
data_set[0], target[0]

(('00-01-0010682284.txt',
  array(['앵커', '불법', '병역', ..., '뉴스', '케이', '이야'], dtype='<U32')),
 '00')

In [17]:
train_data, val_data, train_target, val_target = train_test_split(data_set, target, test_size=0.2, random_state=42)

In [18]:
len(train_data), len(train_target), len(val_data), len(val_target)

(432, 432, 108, 108)

In [19]:
train_data[1], train_target[1], val_data[-1], val_target[-1]

(('00-05-0010726642.txt',
  array(['2013년', '3월', '13일', ..., '뉴스', '스영', '영상'], dtype='<U32')),
 '00',
 ('04-07-0002895933.txt',
  array(['나루히토', '일왕이', '즉위하는', ..., '재배', '배포', '금지'], dtype='<U32')),
 '04')

In [20]:
train_data_category = dict()
for (filename, content), target in zip(train_data, train_target):
    train_data_category[filename] = target

In [21]:
# train_data_category

#### Inverted Index : DTM

In [22]:
global_lexicon, global_posting, global_document, dtm = inverted_index_with_tf(train_data)
len(global_lexicon), len(global_posting), len(global_document), len(dtm)

(80464, 306613, 432, 432)

In [23]:
#global_lexicon

In [24]:
#global_posting[3000:3020]

In [25]:
#global_document

#### DTM

In [26]:
#dtm    # DTM은 Inverted Index(색인) 시 함께 만들어짐

#### TDM

In [27]:
tdm = get_tdm_from_dtm(dtm)

In [28]:
#tdm

#### TWM

In [29]:
twm, dtw = tdm2twm(tdm, global_document)

In [30]:
#twm

In [31]:
#dtw

#### Evaluate IDF

In [32]:
global_lexicon_idf, global_document_weight = evaluate_idf(global_lexicon, global_posting, global_document)

In [33]:
#global_lexicon_idf

In [34]:
#global_document_weight

### Validation

#### Naive Bayes Validation

##### Naive Bayes 분류 과정

**학습**

1. 학습 데이터 카테고리별 문서 수 계산 및 사전 확율 계산 = 카테고리 문서 수 / 전체 문서 수
2. 학습 데이터 카테고리별 token 추출(전처리, 토큰 확장) => train_data에 추출되어 있음
3. 학습 데이터 카테고리별 token의 조건부 확률 계산

**분류**

4. Validation 데이터별 token 추출(전처리, 토큰 확장) => val_data에 추출되어 있음
5. Validation 데이터별 카테고리별 조건부 확률 계산 (학습으로 산출된 token의 조건부 확률 합)
6. 조건부 확률이 가장 높은 카테고리 선택 (prediction)

**평가(validation)**

7. Validation 데이터 labels와 prediction 값 비교

**평가(test)**

8. test data에 대해 분류, 평가(4~7번 항목)

##### 사전 확률

In [35]:
train_freq = defaultdict()

for target in train_target:
    if target not in train_freq.keys():
        train_freq[target] = 1
    else:
        train_freq[target] += 1

train_freq.items()

dict_items([('00', 74), ('05', 72), ('03', 76), ('01', 62), ('04', 74), ('02', 74)])

In [36]:
train_data_length = len(train_target)
prior = dict()

for cat, cat_freq in train_freq.items():
    prior[cat] = cat_freq / train_data_length

prior

{'00': 0.1712962962962963,
 '05': 0.16666666666666666,
 '03': 0.17592592592592593,
 '01': 0.14351851851851852,
 '04': 0.1712962962962963,
 '02': 0.1712962962962963}

##### 카테고리별 단어 빈도

In [37]:
tokens = defaultdict(lambda : defaultdict(int))

for data, target in zip(train_data, train_target):
    for token in data[1]:
        tokens[target][token] += 1

In [38]:
# tokens

In [39]:
vocabulary = set()
tokens_count = dict()

for cat, term_freq in tokens.items():
    vocabulary.update(term_freq.keys())
    tokens_count[cat] = sum(term_freq.values())

In [40]:
len(vocabulary)

80464

In [41]:
tokens_count

{'00': 138246,
 '05': 119041,
 '03': 124455,
 '01': 113530,
 '04': 116018,
 '02': 113946}

##### 카테고리별 token의 조건부 확률

In [42]:
condprob = defaultdict(lambda : defaultdict(float))

B = len(vocabulary)

for token in vocabulary:
    for cat in inverse_category.keys():
        condprob[cat][token] = log((tokens[cat][token] + 1) / (tokens_count[cat] + B))

In [43]:
# condprob

##### Validation data Prediction

In [44]:
print(len(val_data), len(val_target))
print(val_data[0])

108 108
('04-10-0000026378.txt', array(['주간동아', '트뤼도', '캐나다', ..., '거진', '진동', '동아'], dtype='<U32'))


In [45]:
val_pred = []

for vd in val_data:
    val_score = dict()

    for cat in inverse_category.keys():
        val_score[cat] = log(prior[cat])

        for token in vd[1]:
            val_score[cat] += condprob[cat][token]

    val_pred.append(max(val_score, key=val_score.get))    # prediction (가장 높은 조건부 확률의 카테고리 선택)

In [46]:
print(val_pred)

['00', '01', '04', '02', '04', '01', '05', '00', '04', '02', '00', '05', '04', '00', '04', '01', '01', '04', '04', '05', '01', '00', '05', '04', '05', '00', '00', '05', '01', '05', '01', '05', '00', '04', '04', '04', '01', '01', '01', '02', '02', '00', '01', '01', '01', '01', '00', '03', '05', '03', '05', '03', '01', '04', '04', '02', '04', '01', '01', '02', '00', '03', '02', '05', '01', '00', '04', '04', '03', '05', '05', '02', '03', '02', '00', '01', '00', '05', '02', '02', '05', '03', '02', '01', '02', '01', '05', '02', '05', '03', '02', '03', '03', '01', '01', '01', '00', '01', '04', '02', '01', '00', '03', '01', '02', '01', '03', '04']


##### Naive Bayes 성능평가

In [47]:
result = confusion_matrix(val_target, val_pred)
result

array([[13,  0,  1,  0,  2,  0],
       [ 1, 21,  1,  1,  0,  4],
       [ 0,  2, 14,  0,  0,  0],
       [ 0,  1,  1, 11,  1,  0],
       [ 1,  1,  0,  0, 14,  0],
       [ 1,  3,  0,  0,  1, 13]])

In [48]:
print(classification_report(val_target, val_pred))

              precision    recall  f1-score   support

          00       0.81      0.81      0.81        16
          01       0.75      0.75      0.75        28
          02       0.82      0.88      0.85        16
          03       0.92      0.79      0.85        14
          04       0.78      0.88      0.82        16
          05       0.76      0.72      0.74        18

   micro avg       0.80      0.80      0.80       108
   macro avg       0.81      0.80      0.80       108
weighted avg       0.80      0.80      0.80       108



### test data 성능평가

#### test data 읽어오기

In [49]:
files = os.listdir("news_classification_testing_data")
files

['경제-20190408160048011.txt',
 '생활문화-20190407n15403.txt',
 'IT과학-20190408n31939.txt',
 '생활문화-20190408113300382.txt',
 '경제-20190408n25064.txt',
 '세계-20190408150242297.txt',
 'IT과학-20190408163613503.txt',
 '세계-20190408n27328.txt',
 '정치-20190408161154495.txt',
 '사회-20190408n29162.txt',
 '사회-20190408162921246.txt',
 '정치-20190408n27206.txt']

In [50]:
test_target = [category[file.split("-")[0]] for file in files]
test_target

['01', '03', '05', '03', '01', '04', '05', '04', '00', '02', '02', '00']

In [51]:
test_collection = []   # 2차원 배열 : [[filename, content]]

for filename in files:
    with open(os.path.join("news_classification_testing_data", filename), "r") as f:
        test_collection.append([filename, f.read()])

In [52]:
# test_collection

In [53]:
test_collection_cleaned = clean_collection(test_collection)
# test_collection_cleaned

In [54]:
test_data = []
corpus_count = len(test_collection_cleaned)

for idx, (filename, corpus) in enumerate(test_collection_cleaned):
    extended_lexicon = extend_lexicon(corpus)
    test_data.append([filename, extended_lexicon])

In [55]:
test_data[0]

['경제-20190408160048011.txt',
 array(['코스피가', '8일', '외국인', ..., '마감', '감했', '했다'], dtype='<U32')]

#### Test Data Prediction

In [56]:
print(len(test_data), len(test_target))
print(test_data[0])

12 12
['경제-20190408160048011.txt', array(['코스피가', '8일', '외국인', ..., '마감', '감했', '했다'], dtype='<U32')]


In [57]:
test_pred = []

for td in test_data:
    test_score = dict()

    for cat in inverse_category.keys():
        test_score[cat] = log(prior[cat])

        for token in td[1]:
            test_score[cat] += condprob[cat][token]

    test_pred.append(max(test_score, key=test_score.get))    # prediction (가장 높은 조건부 확률의 카테고리 선택)

In [58]:
test_pred

['01', '05', '05', '03', '01', '04', '05', '04', '00', '02', '01', '00']

#### 성능 평가

In [59]:
result = confusion_matrix(test_target, test_pred)
result

array([[2, 0, 0, 0, 0, 0],
       [0, 2, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 2, 0],
       [0, 0, 0, 0, 0, 2]])

In [60]:
print(classification_report(val_target, val_pred))

              precision    recall  f1-score   support

          00       0.81      0.81      0.81        16
          01       0.75      0.75      0.75        28
          02       0.82      0.88      0.85        16
          03       0.92      0.79      0.85        14
          04       0.78      0.88      0.82        16
          05       0.76      0.72      0.74        18

   micro avg       0.80      0.80      0.80       108
   macro avg       0.81      0.80      0.80       108
weighted avg       0.80      0.80      0.80       108

